In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Load data
data = pd.read_excel("/content/sample_data/math_model.xlsx")

# Convert string values to numeric in relevant columns
for j in range(1, 7):
    data["δΦ_V" + str(j)] = pd.to_numeric(data["δΦ_V" + str(j)], errors='coerce')

# Handle missing values (choose either removal or filling)
data = data.dropna()  # Remove rows with NaNs
# OR
data = data.fillna(0)  # Fill NaNs with zeros

# Define problem variables
Vs = [LpVariable(f"V{j}", lowBound=0, upBound=1, cat='Continuous') for j in range(len(data))]
# Create the LP problem (ensure this is done only once)
prob = LpProblem("Minimize_Cost", LpMinimize)

# Construct the objective function using values from the DataFrame
prob += lpSum(data.loc[i, "Φ"] * Vs[i] for i in range(len(Vs)))  # Access "Φ" column for phi_value


# Convert "PV" columns to numeric and handle missing values
for j in range(1, 7):
    data["PV" + str(j)] = pd.to_numeric(data["PV" + str(j)], errors='coerce')
data = data.fillna(0)  # Fill NaNs with zeros (adjust as needed)
# Add constraints
target_emissions = 391.5  # 50% of Phi
budget = 2000  # Maximum allowable cost

prob += lpSum(data.loc[i, "Φ"] - data.loc[i, "δΦ_V" + str(j)] * Vs[j - 1] for i in range(len(data)) for j in range(1, 7)) <= target_emissions
prob += sum([data.loc[i, "PV" + str(j)] * Vs[j - 1] for i in range(len(data)) for j in range(1, 7)]) <= budget

# Solve the problem
prob.solve()

1

In [ ]:
# Print the results
print("Status:", LpStatus[prob.status])
# ... (Access and print variable values, objective function value, etc.)

Status: Optimal


In [3]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, value


In [1]:
pip install pandas pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 37.0 MB/s eta 0:00:00


In [2]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

In [19]:
import pandas as pd
import numpy as np

# Read input data from Excel
data = pd.read_excel("/content/sample_data/math_model.xlsx")
phi_values = data["Φ"].tolist()
k = 0.5
target_phi = {r + 1: k * phi for r, phi in enumerate(phi_values)}
delta_phi_values = {}
for v in range(1, 7):  # V solutions 1 to 6
    delta_phi_values[v] = {}
    for r in range(1, 6):  # Assets 1 to 5
        delta_phi_values[v][r] = data.iloc[r - 1, 6 + v]  # Correct column indexing
cost_values = {}
for v in range(1, 7):
    cost_values[v] = data[f"PV{v}"].tolist()

# Dynamic programming implementation
R = 5
E = 4
V = 6
dp_table = [[float("inf")] * int(target_phi[r] + 1) for r in range(1, R + 1)]  # Convert target to integer
dp_table[0][0] = 0  # Base case: cost 0 to achieve target 0
opt_v_table = [[0] * int(target_phi[r] + 1) for r in range(1, R + 1)]
new_target = 0
for r in range(1, R + 1):
    for remaining_target in range(int(target_phi[r] + 1)):  # Convert target to integer for range()
        for v in range(1, V + 1):
            delta_phi = delta_phi_values[v][r]  # Assuming E = 1 for now
            cost = cost_values[v][0]
            if remaining_target >= delta_phi:
                new_cost = cost + dp_table[r - 1][int(remaining_target - delta_phi)]  # Convert to integer
                print("r:", r)
                print("remaining_target:", remaining_target)
                print("dp_table dimensions:", len(dp_table), len(dp_table[0]))  # Use len() for lists
                if remaining_target >= delta_phi and new_target < len(dp_table[0]):  # Ensure new_target is within bounds
                    new_target = remaining_target - delta_phi

else:
    # Handle cases where remaining_target would exceed bounds
    print("Warning: remaining_target would exceed table bounds. Adjusting calculation.")
    # Modify calculation or handle the situation as needed
for r in range(1, R + 1):
    for remaining_target in range(int(target_phi[r] + 1)):
        for v in range(1, V + 1):
            delta_phi = int(delta_phi_values[v][r])  # Cast to integer
            cost = cost_values[v][0]
            if remaining_target >= delta_phi:
                new_target = remaining_target - delta_phi
                if 0 <= new_target < len(dp_table[r - 1]):
                    new_cost = cost + dp_table[r - 1][new_target]
                    if r < len(dp_table) and remaining_target < len(dp_table[r]) and new_cost < dp_table[r][remaining_target]:
                        dp_table[r][remaining_target] = new_cost
                        opt_v_table[r][remaining_target] = v
# Calculate |V(j,r)|
num_combinations = dp_table[R - 1][int(target_phi[R])]

# Extract optimal array
optimal_array = []
r = R - 1  # Adjust the starting index
remaining_target = int(target_phi[R])  # Cast to integer
while r > 0:
    optimal_v = opt_v_table[r][remaining_target]
    optimal_array.append(optimal_v)
    remaining_target -= int(delta_phi_values[optimal_v][r])
    r -= 1
optimal_array.reverse()

import csv
import pandas as pd

# Write results to output CSV
with open("output_results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Number of combinations (|V(j,r)|)", "Optimal Array"])
    writer.writerow([num_combinations, optimal_array])


r: 1
remaining_target: 2
dp_table dimensions: 5 71
r: 1
remaining_target: 3
dp_table dimensions: 5 71
r: 1
remaining_target: 4
dp_table dimensions: 5 71
r: 1
remaining_target: 5
dp_table dimensions: 5 71
r: 1
remaining_target: 6
dp_table dimensions: 5 71
r: 1
remaining_target: 7
dp_table dimensions: 5 71
r: 1
remaining_target: 7
dp_table dimensions: 5 71
r: 1
remaining_target: 8
dp_table dimensions: 5 71
r: 1
remaining_target: 8
dp_table dimensions: 5 71
r: 1
remaining_target: 9
dp_table dimensions: 5 71
r: 1
remaining_target: 9
dp_table dimensions: 5 71
r: 1
remaining_target: 9
dp_table dimensions: 5 71
r: 1
remaining_target: 10
dp_table dimensions: 5 71
r: 1
remaining_target: 10
dp_table dimensions: 5 71
r: 1
remaining_target: 10
dp_table dimensions: 5 71
r: 1
remaining_target: 11
dp_table dimensions: 5 71
r: 1
remaining_target: 11
dp_table dimensions: 5 71
r: 1
remaining_target: 11
dp_table dimensions: 5 71
r: 1
remaining_target: 12
dp_table dimensions: 5 71
r: 1
remaining_target: 1

In [3]:
!pip install pandas numpy


In [ ]:
#Number of Combinations,Optimal Array
106,"[2, 6, 2, 6]"


In [18]:
# Write results to output CSV
with open("output_results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Number of combinations (|V(j,r)|)", "Optimal Array"])
    writer.writerow([num_combinations, optimal_array])


In [10]:
print(df)  # Print the DataFrame to verify its contents


   Number of combinations (|V(j,r)|) Optimal Array
0                              106.0  [2, 6, 2, 6]


In [5]:
import csv
